<a href="https://colab.research.google.com/github/chaenii989/Project_2_Impacts_on_US_Housing/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
# from config import username, password, host, port, database
import datetime as dt

In [2]:
# Read csv
treas_2020_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/2020_daily_treasury.csv')

treas_2021_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/2021_daily_treasary.csv')

average_home_price_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/average_home_price.csv')

lumber_percent_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/lumber_percent_change.csv')

Q_steel_percent_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/quarterly_steel_percent_change.csv')

url1 = (r"https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/2020_daily_treasury.csv")

url2 = (r"https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/2021_daily_treasary.csv")

lumber_steel_df = pd.read_csv(r"https://raw.githubusercontent.com/chaenii989/Project_2_Impacts_on_US_Housing/main/resources/lumber_steel_percent_change.csv")

homeownership_rate_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/homeownership_rate_RSAHORUSQ156S.csv')
monthly_house_supply_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/monthly_supply_houses_us_MSACSR.csv')
new_housing_permits_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/new_housing_permits_PERMITNSA.csv')
authorized_not_started_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/authorized_not_started_AUTHNOTTSA.csv')
authorized_started_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/authorized_started_HOUST.csv')
under_counstruction_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/under_counstruction_UNDCONTSA.csv')
units_completed_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/units_completed_COMPUTNSA.csv')

In [3]:
#Connect to postresql
# engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')
# session = Session(engine)

In [4]:
 #Rename Columns
Q_steel_percent_df = Q_steel_percent_df.rename(columns={"WPU101_PCH":"steel_q_pct_change","DATE":"date"})
lumber_percent_df = lumber_percent_df.rename(columns={"WPU0811_PCH":"lumber_q_pct_change","DATE":"date"})
treas_2020_df = treas_2020_df.rename(columns={"Date":"date","1 Mo":"one_m_2020","2 Mo":"two_m_2020","3 Mo":"three_m_2020","4 Mo":"four_m_2020", "6 Mo":"six_m_2020", "1 Yr": "one_y_2020","2 Yr": "two_y_2020", "3 Yr": "three_y_2020" ,"5 Yr": "five_Y_2020","7 Yr": "seven_y_2020","10 Yr": "ten_y_2020","20 Yr": "twenty_y_2020","30 Yr": "thirty_y_2020"}, inplace=False)
treas_2021_df = treas_2021_df.rename(columns={"Date":"date","1 Mo":"one_m_2021","2 Mo":"two_m_2021","3 Mo":"three_m_2021","4 Mo":"four_m_2021", "6 Mo":"six_m_2021", "1 Yr": "one_y_2021","2 Yr": "two_y_2021", "3 Yr": "three_y_2021" ,"5 Yr": "five_Y_2021","7 Yr": "seven_y_2021","10 Yr": "ten_y_2021","20 Yr": "twenty_y_2021","30 Yr": "thirty_y_2021"}, inplace=False)
treas_2021_df = treas_2021_df.rename(columns={"Date":"date"})

homeownership_rate_df = homeownership_rate_df.rename(columns={"RSAHORUSQ156S":"homeownership_rate","DATE":"date"})
homeownership_rate_df = homeownership_rate_df.set_index("date")
monthly_house_supply_df = monthly_house_supply_df.rename(columns={"MSACSR":"ratio_for_sale_to_sold","DATE":"date"})
monthly_house_supply_df = monthly_house_supply_df.set_index("date")
new_housing_permits_df = new_housing_permits_df.rename(columns={"PERMIT":"new_permits_thousands","DATE":"date"})
new_housing_permits_df = new_housing_permits_df.set_index("date")
authorized_not_started_df = authorized_not_started_df.rename(columns={"AUTHNOTTSA":"units_not_started_thousands","DATE":"date"})
authorized_not_started_df = authorized_not_started_df.set_index("date")
authorized_started_df = authorized_started_df.rename(columns={"HOUST":"units_started_thousands","DATE":"date"})
authorized_started_df = authorized_started_df.set_index("date")
under_counstruction_df = under_counstruction_df.rename(columns={"UNDCONTSA":"units_under_construction_thousands","DATE":"date"})
under_counstruction_df = under_counstruction_df.set_index("date")
units_completed_df = units_completed_df.rename(columns={"COMPUTSA":"units_constructed_thousands","DATE":"date"})
units_completed_df = units_completed_df.set_index("date")

In [5]:
# Join Tables
construction_permit_df = pd.concat([new_housing_permits_df, authorized_not_started_df,
                                    authorized_started_df, under_counstruction_df,
                                    units_completed_df], axis=1)
construction_permit_df = construction_permit_df.reset_index(drop = False)

In [6]:
average_home_price_df= average_home_price_df.loc[(average_home_price_df["DATE"] >= "2020-01-01")]
average_home_price_df = average_home_price_df.rename(columns={"MSPUS":"average_home_price","DATE":"date"})


In [7]:
treas_2020_df = pd.read_csv(url1)
treas_2020_df = treas_2020_df.rename(columns={"Date":"date","1 Mo":"one_m_2020","2 Mo":"two_m_2020","3 Mo":"three_m_2020","4 Mo":"four_m_2020", "6 Mo":"six_m_2020", "1 Yr": "one_y_2020","2 Yr": "two_y_2020", "3 Yr": "three_y_2020" ,"5 Yr": "five_Y_2020","7 Yr": "seven_y_2020","10 Yr": "ten_y_2020","20 Yr": "twenty_y_2020","30 Yr": "thirty_y_2020"}, inplace=False)
treas_2020_df.head()

,date,one_m_2020,two_m_2020,three_m_2020,six_m_2020,one_y_2020,two_y_2020,three_y_2020,five_Y_2020,seven_y_2020,ten_y_2020,twenty_y_2020,thirty_y_2020
0,1/2/20,1.53,1.55,1.54,1.57,1.56,1.58,1.59,1.67,1.79,1.88,2.19,2.33
1,1/3/20,1.52,1.55,1.52,1.55,1.55,1.53,1.54,1.59,1.71,1.80,2.11,2.26
2,1/6/20,1.54,1.54,1.56,1.56,1.54,1.54,1.56,1.61,1.72,1.81,2.13,2.28
3,1/7/20,1.52,1.53,1.54,1.56,1.53,1.54,1.55,1.62,1.74,1.83,2.16,2.31
4,1/8/20,1.50,1.53,1.54,1.56,1.55,1.58,1.61,1.67,1.78,1.87,2.21,2.35


In [8]:
#formatting the date for 2020
treas_2020_df['date'] = pd.to_datetime(treas_2020_df['date']).dt.normalize()
treas_2020_df.head()

,date,one_m_2020,two_m_2020,three_m_2020,six_m_2020,one_y_2020,two_y_2020,three_y_2020,five_Y_2020,seven_y_2020,ten_y_2020,twenty_y_2020,thirty_y_2020
0,2020-01-02,1.53,1.55,1.54,1.57,1.56,1.58,1.59,1.67,1.79,1.88,2.19,2.33
1,2020-01-03,1.52,1.55,1.52,1.55,1.55,1.53,1.54,1.59,1.71,1.80,2.11,2.26
2,2020-01-06,1.54,1.54,1.56,1.56,1.54,1.54,1.56,1.61,1.72,1.81,2.13,2.28
3,2020-01-07,1.52,1.53,1.54,1.56,1.53,1.54,1.55,1.62,1.74,1.83,2.16,2.31
4,2020-01-08,1.50,1.53,1.54,1.56,1.55,1.58,1.61,1.67,1.78,1.87,2.21,2.35


In [9]:
new_2020_df=treas_2020_df.drop(columns=["one_m_2020",	"two_m_2020",	"three_m_2020",	"six_m_2020",	"one_y_2020",	"two_y_2020",	"three_y_2020",	"five_Y_2020","seven_y_2020"])
new_2020_df.head()

,date,ten_y_2020,twenty_y_2020,thirty_y_2020
0,2020-01-02,1.88,2.19,2.33
1,2020-01-03,1.80,2.11,2.26
2,2020-01-06,1.81,2.13,2.28
3,2020-01-07,1.83,2.16,2.31
4,2020-01-08,1.87,2.21,2.35


In [10]:
treas_2021_df = pd.read_csv(url2)
treas_2021_df = treas_2021_df.rename(columns={"Date":"date","1 Mo":"one_m_2021","2 Mo":"two_m_2021","3 Mo":"three_m_2021","4 Mo":"four_m_2021", "6 Mo":"six_m_2021", "1 Yr": "one_y_2021","2 Yr": "two_y_2021", "3 Yr": "three_y_2021" ,"5 Yr": "five_Y_2021","7 Yr": "seven_y_2021","10 Yr": "ten_y_2021","20 Yr": "twenty_y_2021","30 Yr": "thirty_y_2021"}, inplace=False)
treas_2021_df.head()

,date,one_m_2021,two_m_2021,three_m_2021,six_m_2021,one_y_2021,two_y_2021,three_y_2021,five_Y_2021,seven_y_2021,ten_y_2021,twenty_y_2021,thirty_y_2021
0,1/4/21,0.09,0.09,0.09,0.09,0.10,0.11,0.16,0.36,0.64,0.93,1.46,1.66
1,1/5/21,0.08,0.09,0.09,0.09,0.10,0.13,0.17,0.38,0.66,0.96,1.49,1.70
2,1/6/21,0.09,0.09,0.09,0.09,0.11,0.14,0.20,0.43,0.74,1.04,1.60,1.81
3,1/7/21,0.09,0.09,0.09,0.09,0.11,0.14,0.22,0.46,0.78,1.08,1.64,1.85
4,1/8/21,0.08,0.08,0.08,0.09,0.10,0.14,0.24,0.49,0.81,1.13,1.67,1.87


In [11]:
#creating a pandas df out of the parsed data list
treas_2021_df['date'] = pd.to_datetime(treas_2020_df['date']).dt.normalize()
treas_2021_df.head()

,date,one_m_2021,two_m_2021,three_m_2021,six_m_2021,one_y_2021,two_y_2021,three_y_2021,five_Y_2021,seven_y_2021,ten_y_2021,twenty_y_2021,thirty_y_2021
0,2020-01-02,0.09,0.09,0.09,0.09,0.10,0.11,0.16,0.36,0.64,0.93,1.46,1.66
1,2020-01-03,0.08,0.09,0.09,0.09,0.10,0.13,0.17,0.38,0.66,0.96,1.49,1.70
2,2020-01-06,0.09,0.09,0.09,0.09,0.11,0.14,0.20,0.43,0.74,1.04,1.60,1.81
3,2020-01-07,0.09,0.09,0.09,0.09,0.11,0.14,0.22,0.46,0.78,1.08,1.64,1.85
4,2020-01-08,0.08,0.08,0.08,0.09,0.10,0.14,0.24,0.49,0.81,1.13,1.67,1.87


In [12]:
new_2021_df=treas_2021_df.drop(columns=["one_m_2021",	"two_m_2021",	"three_m_2021",	"six_m_2021",	"one_y_2021",	"two_y_2021",	"three_y_2021",	"five_Y_2021","seven_y_2021"])
new_2021_df.head()

,date,ten_y_2021,twenty_y_2021,thirty_y_2021
0,2020-01-02,0.93,1.46,1.66
1,2020-01-03,0.96,1.49,1.70
2,2020-01-06,1.04,1.60,1.81
3,2020-01-07,1.08,1.64,1.85
4,2020-01-08,1.13,1.67,1.87


In [13]:
# Create tables
average_home_price_df.to_csv('etl/average_home_price.csv', index=False)
Q_steel_percent_df.to_csv('etl/steel_q_pct_change.csv', index=False)
lumber_percent_df.to_csv('etl/lumber_q_pct_change.csv', index=False)
new_2020_df.to_csv('etl/new_2020.csv', index=False)
new_2021_df.to_csv('etl/new_2021.csv', index=False)
lumber_steel_df.to_csv('etl/lumber_steel_percent_change.csv', index=False)

homeownership_rate_df.to_csv('etl/homeownership_rate.csv', index=False)
monthly_house_supply_df.to_csv('etl/monthly_house_supply.csv', index=False)
construction_permit_df.to_csv('etl/construction_permit.csv', index=False)

In [14]:
lumber_steel_df

,date,lumber_prc_change,steel_prc_change
0,4/1/2020,-1.57628,-3.18139
1,7/1/2020,45.00441,-0.85009
2,10/1/2020,1.78336,7.04040
3,1/1/2021,24.50971,23.36722
4,4/1/2021,27.05685,25.68361
